In [1]:
import pandas as pd
import numpy as np

In [2]:
df_raw = pd.read_excel("../data/TABLA_ACCIDENTES_22.xlsx", sheet_name=0) #el excel tiene dos hojas pero solo queremos la primera

df_raw.head()

,ID_ACCIDENTE,ANYO,MES,DIA_SEMANA,HORA,COD_PROVINCIA,COD_MUNICIPIO,ISLA,ZONA,ZONA_AGRUPADA,...,PRIORI_OTRA,CONDICION_NIVEL_CIRCULA,CONDICION_FIRME,CONDICION_ILUMINACION,CONDICION_METEO,CONDICION_NIEBLA,CONDICION_VIENTO,VISIB_RESTRINGIDA_POR,ACERA,TRAZADO_PLANTA
0,1,2022,1,6,3,1,0,NaN,1,1,...,999,1,1,6,1,NaN,NaN,1,998,3
1,2,2022,1,5,16,1,1059,NaN,1,1,...,999,1,3,1,3,NaN,NaN,1,998,2
2,3,2022,1,6,21,1,0,NaN,1,1,...,0,1,1,6,2,NaN,NaN,1,998,2
3,4,2022,1,7,12,1,0,NaN,1,1,...,999,1,3,1,3,NaN,NaN,1,998,1
4,5,2022,1,7,12,1,0,NaN,1,1,...,999,1,1,1,1,NaN,NaN,1,998,3


In [3]:
#Esto es solo para no tener que cargar el dataframe cada vez que nos equivoquemos en algo, con ejecutar esta línea vale
df = df_raw

In [4]:
df.shape

(97916, 73)

El Dataframe del que se parte contiene 97916 filas (registros) y 17 columnas (datos distintos)

In [5]:
df.duplicated().sum()

np.int64(0)

No hay filas duplicadas en el Dataset

In [6]:
df.columns

Index(['ID_ACCIDENTE', 'ANYO', 'MES', 'DIA_SEMANA', 'HORA', 'COD_PROVINCIA',
       'COD_MUNICIPIO', 'ISLA', 'ZONA', 'ZONA_AGRUPADA', 'CARRETERA', 'KM',
       'SENTIDO_1F', 'TITULARIDAD_VIA', 'TIPO_VIA', 'TIPO_ACCIDENTE',
       'TOTAL_MU24H', 'TOTAL_HG24H', 'TOTAL_HL24H', 'TOTAL_VICTIMAS_24H',
       'TOTAL_MU30DF', 'TOTAL_HG30DF', 'TOTAL_HL30DF', 'TOTAL_VICTIMAS_30DF',
       'TOTAL_VEHICULOS', 'TOT_PEAT_MU24H', 'TOT_BICI_MU24H',
       'TOT_CICLO_MU24H', 'TOT_MOTO_MU24H', 'TOT_TUR_MU24H', 'TOT_FURG_MU24H',
       'TOT_CAM_MENOS3500_MU24H', 'TOT_CAM_MAS3500_MU24H', 'TOT_BUS_MU24H',
       'TOT_OTRO_MU24H', 'TOT_SINESPECIF_MU24H', 'TOT_PEAT_MU30DF',
       'TOT_BICI_MU30DF', 'TOT_CICLO_MU30DF', 'TOT_MOTO_MU30DF',
       'TOT_TUR_MU30DF', 'TOT_FURG_MU30DF', 'TOT_CAM_MENOS3500_MU30DF',
       'TOT_CAM_MAS3500_MU30DF', 'TOT_BUS_MU30DF', 'TOT_VMP_MU30DF',
       'TOT_OTRO_MU30DF', 'TOT_SINESPECIF_MU30DF', 'NUDO', 'NUDO_INFO',
       'CARRETERA_CRUCE', 'PRIORI_NORMA', 'PRIORI_AGENTE', 'PR

Hay hasta 31 columnas indicando la gravedad del accidente, por lo que lo primero que haremos será agruparlas todas en una donde tan solo se indique la gravedad como leve, grave o mortal.

Lo primero para ello es entender nuestros datos, por ejemplo: 
    
    ¿La columna que indica el número de fallecidos computados a 30 días integra los fallecidos computados a 24 horas o son independientes?
    ¿La columna que indica el número de fallecidos computados a 30 días integra las victimas computadas a 24 horas o son independientes?

In [7]:
print(f"Total de fallecidos registrados en el accidente computados a 24 horas mas que computados a 30 días es: {df.loc[df['TOTAL_MU24H'] > df['TOTAL_MU30DF']].shape[0]} fallecidos \n")

print(f"Total de víctimas registrados en el accidente computados a 24 horas mas que computados a 30 días es: {df.loc[df['TOTAL_VICTIMAS_24H'] > df['TOTAL_VICTIMAS_30DF']].shape[0]} victimas")

Total de fallecidos registrados en el accidente computados a 24 horas mas que computados a 30 días es: 0 fallecidos 

Total de víctimas registrados en el accidente computados a 24 horas mas que computados a 30 días es: 0 victimas


No existe ningun accidente con más fallecidos ni victimas computadas a 24 horas que a 30 días, se asume por tanto, que el registro computado a 30 días ya incluye el de 24 horas.. 


No vamos a eliminar el resto de columnas sobre fallecidos ya que aunque no se vayan a incluir en nuestro modelo pueden ser interesantes para analizar estadísticas.

Otra cuestión parecida a la anterior es si el número de heridos hospitalizados incluye también a los que acabaron muriendo. Este problema es más complicado, ya que si una víctima es hospitalizada y acaba muriendo en los 30 días posteriores es posible que cuente dentro de los dos grupos, no obstante dado que el total de victimas se calcula con la suma de fallecidos, hospitalizados y no hospitalizados, en teoría si alguien acaba falleciendo en el hospital no debería contar como victima hospitalizada ya que sino al hacer la suma estarían contando doble las victimas en estos casos, pero vamos a comprobarlo.

In [8]:
df[['TOTAL_MU24H','TOTAL_MU30DF','TOTAL_HG24H','TOTAL_HG30DF']].loc[df['TOTAL_MU30DF'] > df['TOTAL_HG30DF']]

,TOTAL_MU24H,TOTAL_MU30DF,TOTAL_HG24H,TOTAL_HG30DF
16,1,1,0,0
133,1,1,0,0
196,1,1,0,0
324,0,1,1,0
446,2,2,0,0
...,...,...,...,...
97610,1,1,0,0
97619,1,1,0,0
97622,1,1,0,0
97646,0,1,1,0


Como se puede comprobar hay columnas en las que el número de fallecidos en 30 días supera al de ingresados en el mismo periodo, aunque pudiera ser que fallecieron fuera sin ser ingresados. 

Pero filas como la 16 donde hay dos víctimas, y una falleció en 24 horas y la otra en los 30 días indican que en efecto la columna de fallecidos es independiente de la de hospitalizados en el mismo periodo.

In [9]:
#más hospitalizados en 24 horas que en 30 días
df[['TOTAL_MU24H','TOTAL_MU30DF','TOTAL_HG24H', 'TOTAL_HG30DF','TOTAL_HL24H','TOTAL_HL30DF','TOTAL_VICTIMAS_24H','TOTAL_VICTIMAS_30DF']].loc[df['TOTAL_HG24H'] > df['TOTAL_HG30DF']]

,TOTAL_MU24H,TOTAL_MU30DF,TOTAL_HG24H,TOTAL_HG30DF,TOTAL_HL24H,TOTAL_HL30DF,TOTAL_VICTIMAS_24H,TOTAL_VICTIMAS_30DF
324,0,1,1,0,0,0,1,1
605,0,1,1,0,0,0,1,1
620,0,1,4,3,0,0,4,4
1568,0,1,1,0,2,2,3,3
1763,0,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...
97226,0,1,1,0,0,0,1,1
97229,0,1,1,0,0,0,1,1
97506,0,1,1,0,0,0,1,1
97646,0,1,1,0,0,0,1,1


Es curioso que si que haya algunos casos donde hay más hospitalizados en 24 horas que en 30 días, por lo que a priori parece que en este caso estár hospitalizado en las primeras 24 horas no implica contar como que estuviste hospitalizado en los 30 días posteriores, no obstante hay algunas incoherencias:
- En todos los casos hubo un fallecido en 30 días y ninguno en 24 horas (Cuando solo era una victima)
- Si estuvo hospitalizado menos de 24 horas y no falleció en esas horas cabría pensar que se trata de un accidente leve, pero solo hay 232 casos cuando los accidentes leves deberían ser mayoritarios, además falleció en los siguientes 30 días lo que indica que fue un accidente mortal.

Teoría: Como hemos mencionado anteriormente alguien fallecido en 30 días no será incluido en la columna de hospitalizados en ese periodo ya que generaría un conteo doble en el total de víctimas, no obstante, si será incluido en los hospitalizados en 24 horas ya que eso no genera ningúna incoherencia. Por tanto, los casos que observamos en la tabla anterior son casos donde la victima del accidente fue llevada al hospital de inmediato (Hospitalizado 24 horas), no falleció en las primeras 24 horas pero sí falleció en los siguientes días y lo sacaron de la lista de hospitalizados en 30 días. En estos casos la víctima nunca fallece en las primeras 24 horas porque de hacerlo se le sacaría del número de hospitalizados en ese periodo (y en el siguiente también ya que contaría como fallecido).

Vamos a confirmar esta teoría, para ello se tienen que dar varias condiciones, que son:
- 1º Si el número de fallecidos en 24 horas es 0 y el número de fallecidos en 30 días es igual al de ingresados en 24 horas y mayor a 0, el número de ingresados en 30 días nunca podrá ser ni igual ni mayor al de fallecidos en 30 días.

In [10]:
# 1º Condición
print(f"Casos que no cumplen la primera condición: {df[['TOTAL_MU24H','TOTAL_MU30DF','TOTAL_HG24H', 'TOTAL_HG30DF','TOTAL_VICTIMAS_24H',
       'TOTAL_VICTIMAS_30DF','TOTAL_HL24H','TOTAL_HL30DF']].loc[(df['TOTAL_MU24H'] == 0) & (df['TOTAL_MU30DF']==df['TOTAL_HG24H']) & (df['TOTAL_MU30DF']>0) & (df['TOTAL_HG30DF']>=df['TOTAL_MU30DF'])].shape[0]} casos")


Casos que no cumplen la primera condición: 0 casos


- 2º En los casos donde el número de fallecidos computados a 24 horas y 30 días sea el mismo, y el número de ingresados computados a 24 horas y 30 días también lo sea, serán siempre casos en donde hubo más de una víctima (al menos una que falleció y una que sobrevivió)

In [11]:
#2º Condición
df[['TOTAL_MU24H','TOTAL_MU30DF','TOTAL_HG24H', 'TOTAL_HG30DF',
       'TOTAL_VICTIMAS_30DF']].loc[(df['TOTAL_MU24H'] == df['TOTAL_MU30DF']) & (df['TOTAL_HG24H']==df['TOTAL_MU30DF']) & (df['TOTAL_HG24H']==1)]


,TOTAL_MU24H,TOTAL_MU30DF,TOTAL_HG24H,TOTAL_HG30DF,TOTAL_VICTIMAS_30DF
975,1,1,1,1,2
1303,1,1,1,1,2
1324,1,1,1,1,2
1332,1,1,1,1,2
1379,1,1,1,1,2
...,...,...,...,...,...
93982,1,1,1,1,2
94088,1,1,1,1,4
94451,1,1,1,1,2
97596,1,1,1,1,2


- 3º El total de víctimas en computados a 24 horas y computados a 30 días siempre será el mismo.

In [12]:
#3º Condición
print(f"Número de víctimas registradas computadas a 24 horas diferentes a computadas a 30 días: {df.loc[df['TOTAL_VICTIMAS_24H'] != df['TOTAL_VICTIMAS_30DF']].shape[0]} victimas")

Número de víctimas registradas computadas a 24 horas diferentes a computadas a 30 días: 0 victimas


Todas las condiciones se han cumplido, por lo que parece que la teoría planteada era correcta. Además se termina de concluir que los registros computados a 30 días incluyen los computados a 24 horas.

In [13]:
# ALEX: Este creo que no aporta nada pero lo he dejado por si opinas lo contrario, si estás de acuerdo bórralo
df[['TOTAL_MU30DF','TOTAL_HG30DF','TOTAL_HL30DF','TOTAL_VICTIMAS_30DF']].loc[(df['TOTAL_MU30DF'] == df['TOTAL_HG30DF']) & (df['TOTAL_MU30DF'] > 1)]

,TOTAL_MU30DF,TOTAL_HG30DF,TOTAL_HL30DF,TOTAL_VICTIMAS_30DF
12034,2,2,2,6
12851,2,2,0,4
29576,2,2,6,10
30031,2,2,2,6
31432,3,3,0,6
83079,2,2,0,4
88434,2,2,6,10
91138,2,2,0,4


In [14]:
#Aquí habría que ver si borramos la de no hospitalizados en 24 horas, nos sirve de algo?
df = df.drop(['TOTAL_MU24H','TOTAL_VICTIMAS_24H'], axis=1)

Diferenciaremos entre tres tipos de accidentes: Mortal, leve y sin lesiones. Ya que en el dataset no se indica el tipo de lesión vamos a establecer el siguiente criterio:

- Mortal = La víctima fallece (en cualquier periodo).
- Leve = Fue al hospital
- Sin lesiones = No fue al hospital

Se establece la columna tipo de accidente en función de la prioridad:

        Si hay una victima mortal, el accidente se clasifica como  'Mortal'
        Si no hay ningun accidente mortal pero si ha sigo ingresado el accidente se clasifica como 'Leve'
        Si no hay ningun accidente mortal ni nadie ha sido ingresado el accidente se clasifica como 'Sin_lesiones'

In [15]:

def clasificar_accidentes(column):
    if column['TOTAL_MU30DF'] > 0:
        return 'Mortal'
    elif column['TOTAL_HG30DF'] > 0 & column['TOTAL_MU30DF'] == 0: 
        return 'Leve'
    else: 
        return 'Sin_lesiones'

df['TYP_ACC'] = df.apply(clasificar_accidentes, axis=1) # Crear la columna TYP_ACC basada en las condicione



Se contabiliza el número de accidentes mortales, leves y sin lesiones que hay en el dataset

In [16]:
cont_mortales = 0
cont_leves = 0
cont_sin_lesiones = 0

for i in df['TYP_ACC']:
    if i == "Mortal":
        cont_mortales += 1
    elif i == "Leve":
        cont_leves += 1
    else:
        cont_sin_lesiones +=1

total_accidentes = cont_mortales + cont_leves + cont_sin_lesiones
porc_mortales = (cont_mortales/total_accidentes)*100
porc_leves = (cont_leves/total_accidentes)*100
porc_sin_lesiones = (cont_sin_lesiones/total_accidentes)*100

print(f"El numero de accidentes mortales es {cont_mortales} un {porc_mortales:.2f}% del total de accidentes\n")
print(f"El número de accidentes leves es {cont_leves} un {porc_leves:.2f}% del total de accidentes \n")
print(f"El número de accidentes sin lesiones es {cont_sin_lesiones} un {porc_sin_lesiones:.2f}% del total de accidentes")


El numero de accidentes mortales es 1620 un 1.65% del total de accidentes

El número de accidentes leves es 7410 un 7.57% del total de accidentes 

El número de accidentes sin lesiones es 88886 un 90.78% del total de accidentes


Se extrae que la gran mayoría de accidentes han sido sin lesiones, una minoria leves y una gran minoria mortales.

# Limpieza y simplificación del Dataset

A continuación se eliminan las columnas que no se consideran representativas para el estudio y posterior entrenameinto del modelo

In [17]:
# Eliminar irrelevantes
df = df.drop(['ID_ACCIDENTE','ZONA_AGRUPADA','ANYO','TITULARIDAD_VIA','TOT_PEAT_MU24H','TOT_BICI_MU24H','TOT_CICLO_MU24H', 
        'TOT_MOTO_MU24H', 'TOT_TUR_MU24H','TOT_FURG_MU24H','TOT_CAM_MENOS3500_MU24H','TOT_CAM_MAS3500_MU24H', 'TOT_BUS_MU24H',
       'TOT_OTRO_MU24H','TOT_SINESPECIF_MU24H','TOT_BUS_MU30DF','TOT_SINESPECIF_MU30DF',
       'NUDO_INFO','CARRETERA_CRUCE','CONDICION_NIEBLA','CONDICION_VIENTO','TOT_CAM_MENOS3500_MU30DF'], axis=1)


<font color="red">No entiendo a que se refiere la última columna del dataset 'TRAZADDO_PLANTA' la eliminamos? </font>

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97916 entries, 0 to 97915
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   MES                      97916 non-null  int64  
 1   DIA_SEMANA               97916 non-null  int64  
 2   HORA                     97916 non-null  int64  
 3   COD_PROVINCIA            97916 non-null  int64  
 4   COD_MUNICIPIO            97916 non-null  int64  
 5   ISLA                     5207 non-null   float64
 6   ZONA                     97916 non-null  int64  
 7   CARRETERA                97916 non-null  object 
 8   KM                       50944 non-null  float64
 9   SENTIDO_1F               97916 non-null  int64  
 10  TIPO_VIA                 97916 non-null  int64  
 11  TIPO_ACCIDENTE           97916 non-null  int64  
 12  TOTAL_HG24H              97916 non-null  int64  
 13  TOTAL_HL24H              97916 non-null  int64  
 14  TOTAL_MU30DF          

Parece que el dataset no tiene valores nulos excepto por la columna 'ISLA' que es normal porque la mayoría de accidentes ocurrieron en el territorio peninsular y en 'KM' que se trata de carreteras no inventariadas. No obstante esto no es cierto ya que en el dataset se utiliza 999 para indicar los valores faltantes, por eso parece que no hay nulos, vamos a eliminar estos valores para ver realmente como es nuestro Dataframe. También vamos a sustituir los valores nulos en 'ISLA' con 0 para indicar que no aplica.

In [19]:
df['ISLA'] = df['ISLA'].fillna(0) #Rellenar columna ISLA

#Gestionar nulos en columnas prioridad
cols_prioridad = ['PRIORI_NORMA', 'PRIORI_AGENTE', 'PRIORI_SEMAFORO', 'PRIORI_VERT_STOP',
                  'PRIORI_VERT_CEDA', 'PRIORI_HORIZ_STOP', 'PRIORI_HORIZ_CEDA', 'PRIORI_MARCAS',
                  'PRIORI_PEA_NO_ELEV', 'PRIORI_PEA_ELEV', 'PRIORI_MARCA_CICLOS',
                  'PRIORI_CIRCUNSTANCIAL', 'PRIORI_OTRA']

df.loc[(df['PRIORI_NORMA'] == 999) & (df['NUDO'] == 1), cols_prioridad] = df.loc[(df['PRIORI_NORMA'] == 999) & (df['NUDO'] == 1), cols_prioridad].replace(999, np.nan)

df[['ACERA', 'VISIB_RESTRINGIDA_POR']] = df[['ACERA', 'VISIB_RESTRINGIDA_POR']].replace([999, 18], np.nan)


In [20]:
#Gestionar columna 'KM' muchos valores nulos, valores 0, 999, 9999 que también significan nulo

In [20]:
#Comprobar el número de valores nulos en las columnas ajustadas
df_subset_nulls = df[['ISLA'] + cols_prioridad + ['ACERA', 'VISIB_RESTRINGIDA_POR']]
df_subset_nulls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97916 entries, 0 to 97915
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ISLA                   97916 non-null  float64
 1   PRIORI_NORMA           91150 non-null  float64
 2   PRIORI_AGENTE          91150 non-null  float64
 3   PRIORI_SEMAFORO        91150 non-null  float64
 4   PRIORI_VERT_STOP       91150 non-null  float64
 5   PRIORI_VERT_CEDA       91150 non-null  float64
 6   PRIORI_HORIZ_STOP      91150 non-null  float64
 7   PRIORI_HORIZ_CEDA      91150 non-null  float64
 8   PRIORI_MARCAS          91150 non-null  float64
 9   PRIORI_PEA_NO_ELEV     91150 non-null  float64
 10  PRIORI_PEA_ELEV        91150 non-null  float64
 11  PRIORI_MARCA_CICLOS    91150 non-null  float64
 12  PRIORI_CIRCUNSTANCIAL  91150 non-null  float64
 13  PRIORI_OTRA            91150 non-null  float64
 14  ACERA                  94950 non-null  float64
 15  VI

Aunque hay más nulos que antes, siguen siendo pocos teniendo en cuenta el tamaño del dataset, por lo que no eliminaremos ninguna columna por este criterio.

A continuación se combinan tipos de accidente 

In [21]:
def tipos_accidentes(column):
    if (column['TIPO_ACCIDENTE'] > 0) & (column['TIPO_ACCIDENTE'] < 7):
        return 'Colisión'
    elif (column['TIPO_ACCIDENTE'] > 6) & (column['TIPO_ACCIDENTE'] < 9): 
        return 'Atropello'
    elif (column['TIPO_ACCIDENTE'] >10) & (column['TIPO_ACCIDENTE'] < 20):
        return 'Salida de vía'
    else:
        return 'Otro tipo de accidente'

df['TIPO_ACCIDENTE'] = df.apply(tipos_accidentes, axis=1) 


In [22]:
def condicion_firme(column):
    if (column['CONDICION_FIRME'] > 0) & (column['CONDICION_FIRME'] < 2):
        return 'Seco y limpio'
    elif (column['CONDICION_FIRME'] > 1) & (column['CONDICION_FIRME'] < 3): 
        return 'Con barro o gravilla suelta'
    elif (column['CONDICION_FIRME'] >2) & (column['CONDICION_FIRME'] < 5):
        return 'Mojado'
    elif (column['CONDICION_FIRME'] >4) & (column['CONDICION_FIRME'] < 8):
        return 'Resbaladizo'
    else:
        return 'Se desconoce'

df['CONDICION_FIRME'] = df.apply(condicion_firme, axis=1) 

Hay ciertas variables que aunque son categóricas como la provincia, el tipo de vía, el tipo de accidente, etc. Se encuentran en formato numérico, lo cual para entrenar el modelo será mejor pero para visualizar los datos puede ser muy confuso, por lo que vamos a crear diccionarios de todas ellas para ayudar visualizar los datos.

In [23]:
#Diccionarios

dia_semana = {1:'Lunes', 2:'Martes', 3:'Miercoles', 4:'Jueves', 5:'Viernes', 6:'Sábado', 7:'Domingo'}

cod_provincia = {1: 'Álava', 2: 'Albacete', 3: 'Alicante', 4: 'Almería', 5: 'Ávila', 6: 'Badajoz', 7: 'Islas Baleares', 8: 'Barcelona', 
                9: 'Burgos', 10: 'Cáceres', 11: 'Cádiz', 12: 'Castellón', 13: 'Ciudad Real', 14: 'Córdoba', 15: 'Coruña', 16: 'Cuenca', 
                17: 'Girona', 18: 'Granada', 19: 'Guadalajara', 20: 'Gipuzkoa', 21: 'Huelva', 22: 'Huesca', 23: 'Jaén', 24: 'León', 
                25: 'Lleida', 26: 'Rioja, La', 27: 'Lugo', 28: 'Madrid', 29: 'Málaga', 30: 'Murcia', 31: 'Navarra', 32: 'Ourense', 
                33: 'Asturias', 34: 'Palencia', 35: 'Las Palmas', 36: 'Pontevedra', 37: 'Salamanca', 38: 'Santa Cruz de Tenerife', 
                39: 'Cantabria', 40: 'Segovia', 41: 'Sevilla', 42: 'Soria', 43: 'Tarragona', 44: 'Teruel', 45: 'Toledo', 46: 'Valencia', 
                47: 'Valladolid', 48: 'Bizkaia', 49: 'Zamora', 50: 'Zaragoza', 51: 'Ceuta', 52: 'Melilla'}

#Los códigos de municipios se encuentran en un excel por lo que primero hay que abrirlos y después crear el diccionario
df_municipios = pd.read_excel('../data/diccionario_municipio.xlsx', header=0, converters= {
    'COD': lambda x: str(x) #Convertir a texto para que no se eliminen los 0
})
cod_municipio = dict(zip(df_municipios['COD'], df_municipios['NOMBRE '])) #crear diccionario

FileNotFoundError: [Errno 2] No such file or directory: '../data/diccionario_municipio.xlsx'

In [ ]:
# https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177031&menu=ultiDatos&idp=1254734710990 codigos municipio

In [24]:
df.columns

Index(['MES', 'DIA_SEMANA', 'HORA', 'COD_PROVINCIA', 'COD_MUNICIPIO', 'ISLA',
       'ZONA', 'CARRETERA', 'KM', 'SENTIDO_1F', 'TIPO_VIA', 'TIPO_ACCIDENTE',
       'TOTAL_HG24H', 'TOTAL_HL24H', 'TOTAL_MU30DF', 'TOTAL_HG30DF',
       'TOTAL_HL30DF', 'TOTAL_VICTIMAS_30DF', 'TOTAL_VEHICULOS',
       'TOT_PEAT_MU30DF', 'TOT_BICI_MU30DF', 'TOT_CICLO_MU30DF',
       'TOT_MOTO_MU30DF', 'TOT_TUR_MU30DF', 'TOT_FURG_MU30DF',
       'TOT_CAM_MAS3500_MU30DF', 'TOT_VMP_MU30DF', 'TOT_OTRO_MU30DF', 'NUDO',
       'PRIORI_NORMA', 'PRIORI_AGENTE', 'PRIORI_SEMAFORO', 'PRIORI_VERT_STOP',
       'PRIORI_VERT_CEDA', 'PRIORI_HORIZ_STOP', 'PRIORI_HORIZ_CEDA',
       'PRIORI_MARCAS', 'PRIORI_PEA_NO_ELEV', 'PRIORI_PEA_ELEV',
       'PRIORI_MARCA_CICLOS', 'PRIORI_CIRCUNSTANCIAL', 'PRIORI_OTRA',
       'CONDICION_NIVEL_CIRCULA', 'CONDICION_FIRME', 'CONDICION_ILUMINACION',
       'CONDICION_METEO', 'VISIB_RESTRINGIDA_POR', 'ACERA', 'TRAZADO_PLANTA',
       'TYP_ACC'],
      dtype='object')

# Ideas: 


<font color="red">Para poner mensajes en rojo </font>

<font color="red">Relación entre el tipo de accidente y la visibilidad del trazado </font>

<font color="red">Tipo de accidente y provincia </font>
